In [ ]:
import os, glob, shutil, cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from scipy.stats import linregress
from skimage.filters import threshold_otsu, threshold_yen, try_all_threshold, threshold_minimum, threshold_isodata
import skimage.measure as measure

# Extracting frames from video using cv2 package

In [ ]:
#find videos
print(f'Current directory: {os.getcwd()}')

path_to_videos = glob.glob(os.path.join('Videos','*.avi'))
print(path_to_videos, '-'*10, sep = '\n')
#create required directories
def makefolders(path):
    folder = os.path.split(path) #create folders for extracting all images
    folder = os.path.splitext(folder[-1])
    folder_create = os.path.join('Videos',folder[0])
    if os.path.exists(folder_create) == False:
        os.mkdir(folder_create)
    else:
        pass
    return folder_create

#navigate into folder for the image and save images
def saveimages(path, dir_name):
    cam = cv2.VideoCapture(path)
    count, success = 0, True
    video_index = []
    while success:
        success, image = cam.read()
        if not cam.isOpened():
            print(f"Error: Could not open video file {path}")
        os.chdir(dir_name)
        if success:
            folder = os.path.split(dir_name)
            frame = f'{folder[-1]}_frame_{count:02}.jpg'
            video_index.append(frame)
            if os.path.exists(frame) == False:
                cv2.imwrite(frame, image)
                print(f'{frame} successfully saved')
                count+=1
            else:
                print(f'{frame} already exists, skipping save')
                count+=1
        else:
            print('End of file')
        os.chdir(os.path.join('..','..'))
    cam.release()
    print(f'Current directory: {os.getcwd()} \n Completed {path}\n--------------------')
    return video_index

In [ ]:
video_indexes, folder_indexes = [], []
vf_dict = {}
for path in path_to_videos:
    dir_name = makefolders(path)
    folder_indexes.append(dir_name)
    img_names = saveimages(path, dir_name)
    video_indexes.append(img_names)
    vf_dict[os.path.split(dir_name)[-1]] = img_names

#show structure of the dictionary
for key, item in vf_dict.items():
    print(key, item, '\n')

# Process fluorescence data - First pass

In [ ]:
# Functions for extracting and plotting the raw data
cwd = os.getcwd() #getting current working directory

folders_to_process = ('LipoKDEL1ug_biotin_no drug_live_5min1h',  'LipoKDEL1ug_biotin_drug25uM_live_5min1h',
                      'XtremeStr-li1.5ug_biotin_drug50uM_live_5min1h',
                      '0.5ugDNA_drug10uM_5min45min_1', '0.75ugDNA_drug10uM_5min45min_1', '1ugDNA_drug10uM_5min45min_1')

region_of_interest = {'0.5ugDNA_drug10uM_5min45min_1': [(300,500, 400,600)],
           '0.75ugDNA_drug10uM_5min45min_1': [(300,475, 375,550),(690,780, 400,500), (530,630, 530,630)],
           '1ugDNA_drug10uM_5min45min_1': [(400,550, 180,300),(695,790, 655,725)],
           'LipoKDEL1ug_biotin_drug25uM_live_5min1h': [(625,740, 600,720)],
           'LipoKDEL1ug_biotin_no drug_live_5min1h': [(550,690, 450,600)],
           'XtremeStr-li1.5ug_biotin_drug50uM_live_5min1h': [(285,400, 70,150),(615,675, 650,715), (170,260, 970,1024)]
}

golgi_frame = {'0.5ugDNA_drug10uM_5min45min_1': '0.5ugDNA_drug10uM_5min45min_1_frame_03.jpg',
           '0.75ugDNA_drug10uM_5min45min_1': '0.75ugDNA_drug10uM_5min45min_1_frame_02.jpg',
           '1ugDNA_drug10uM_5min45min_1': '1ugDNA_drug10uM_5min45min_1_frame_03.jpg',
           'LipoKDEL1ug_biotin_drug25uM_live_5min1h': 'LipoKDEL1ug_biotin_drug25uM_live_5min1h_frame_06.jpg',
           'LipoKDEL1ug_biotin_no drug_live_5min1h': 'LipoKDEL1ug_biotin_no drug_live_5min1h_frame_04.jpg',
           'XtremeStr-li1.5ug_biotin_drug50uM_live_5min1h': 'XtremeStr-li1.5ug_biotin_drug50uM_live_5min1h_frame_06.jpg'
              }

#creating a function called extract_fluorescenceData, to loop through the 7 different folders.
def extract_fluorescenceData(folder_path, img_order, resolutions):
    total_fluorescence = [] #empty list to add the max fluoresence later and plot
    subdir_path = os.path.join(cwd,'Videos', folder_path) #create path to directory in the iteration

    if os.path.isdir(subdir_path): #checking if subdirectory path exist
        for img in img_order:
            img_path = os.path.join(subdir_path, img)
            if os.path.exists(img_path): #filter for only .jpg files inside subdir_path
                a=plt.imread(img_path) #need save plt.imread to a variable, if not cannot plot downstream
                img_gray = rgb2gray(a) #converting the img into grayscale
                y1, y2, x1, x2 = resolutions
                cropped_img = img_gray[y1:y2, x1:x2]
                total_fluorescence.append(cropped_img.sum()) #sum up signal inside individual frames and append into total_fluorascence
    else:
        print(f'Subdirectory {video_folders} not found')
    return total_fluorescence #return here so can use in "histogrammer" function. this variable will be 'video1' in histogrammer function

def histogrammer(video1, n, i=0, label = False): #video1 is an array of the max fluorescence values, i simply stands for index
    video1_normalise = video1/max(video1) #normalising to highest sum fluorescent signal in the frames.
    name = os.path.split(n) #split it by \\ into index, where [0] = your directory and [i] in this case is folder names
    x = range(1, len(video1)+1) #plotting frame 1 to n+1

    #plotting segment
    if label == True:
        plt.plot(x, video1_normalise, linestyle = 'dashed', marker = 'o', label = 'Original Data') #marker is the data point, marked as circle.
    else:
        plt.plot(x, video1_normalise, linestyle = 'dashed', marker = 'o', label = f'{name} ROI {i}')
    #Labels & aesthetics
    # plt.xticks(x) #this makes it show every tick in X axis
    plt.xlabel('Frame number', fontsize = 10)
    plt.ylabel('Total fluorescence signal', fontsize = 10)
    plt.grid(alpha = .25)
    plt.tight_layout()


In [ ]:
longest_video = 0
plt.figure(figsize =(20,6))
for key in region_of_interest.keys():
    for i in range(len(region_of_interest[key])):
        video1 = extract_fluorescenceData(key, vf_dict[key], region_of_interest[key][i])
        if longest_video < len(video1):
            longest_video = len(video1)
        histogrammer(video1, key, i)
plt.title('Graph of all regions of interest', fontsize = 10)
plt.xticks(np.arange(1,longest_video+1))
plt.legend(loc = 'upper right', bbox_to_anchor = (1.5,1))
plt.tight_layout()
plt.savefig(f'Graph of all regions of interest', format = 'jpg')
plt.show()

# Attempting to assess the cropping of the images

In [ ]:
#goal: identify green regions and calculate area to intensity ratio for lipoKDEL1ug_biotin_drug25uM
video = {'LipoKDEL1ug_biotin_drug25uM_live_5min1h': (625,740, 600,720)}

img_path = 'LipoKDEL1ug_biotin_drug25uM_live_5min1h_frame_03.jpg'
img = plt.imread(os.path.join('Videos',list(video.keys())[0], img_path))
img = img[625:740,600:720]

plt.imshow(img)

img_grey = rgb2gray(img)
plt.show()

In [ ]:
threshold = threshold_minimum(img_grey)
img_binarised = img_grey < threshold

img_labelled = measure.label(img_binarised.astype('uint8'))
plt.imshow(img_labelled)
 # measure.label() requires an image of type int
img_labelled = measure.label(img_binarised.astype('uint8'))
region_info = measure.regionprops(img_labelled)
print(region_info[0].area)

no_of_regions = len(region_info)

for count, region in enumerate(region_info):
    print('-'*10, f'Region {count}', '-'*10)
    print(f'Centre\t: {region.centroid}')
    print(f'Area\t: {region.area}')
    print('\n')

In [ ]:
img_masked = img_labelled == 1
plt.imshow(img_masked)
plt.show()

In [ ]:
def ratio_Data(folder_path, img_order, resolutions): #attempting to find fluorescence/area ratio
    total_fluorescence, areas = [], []
    subdir_path = os.path.join(cwd,'Videos', folder_path) #create path to directory in the iteration

    if os.path.isdir(subdir_path): #checking if subdirectory path exist
        for img in img_order:
            img_path = os.path.join(subdir_path, img)
            if os.path.exists(img_path):
            # if img.lower().endswith('.jpg'): #filter for only .jpg files inside subdir_path
                a=plt.imread(img_path) #need save plt.imread to a variable, if not cannot plot downstream
                img_gray = rgb2gray(a) #converting the img into grayscale
                y1, y2, x1, x2 = resolutions
                cropped_img = img_gray[y1:y2, x1:x2]

                try:
                    threshold = threshold_minimum(cropped_img)
                    binarised_img = cropped_img < threshold
                    img_labelled = measure.label(binarised_img.astype('uint8'))
                    region_info = measure.regionprops(img_labelled)

                    total_fluorescence.append(cropped_img.sum()) #sum up signal inside individual frames and append into total_fluorascence
                    areas.append((y2-y1)*(x2-x1) - region_info[0].area)
                except RuntimeError:
                    print(f'Error with {img} image')
                    pass
    else:
        print(f'Subdirectory {video_folders} not found')
    ratio = np.array(total_fluorescence)/np.array(areas)
    return ratio

def overlay_threshold(folder_path, img_order, resolutions, return_base = True):
    total_fluorescence, areas = [], []
    subdir_path = os.path.join(cwd,'Videos', folder_path) #create path to directory in the iteration
    y1, y2, x1, x2 = resolutions
    overlay = np.zeros((y2-y1,x2-x1))

    if os.path.isdir(subdir_path): #checking if subdirectory path exist
        for img in img_order:
            img_path = os.path.join(subdir_path, img)
            if os.path.exists(img_path):
                a=plt.imread(img_path) #need save plt.imread to a variable, if not cannot plot downstream
                img_gray = rgb2gray(a) #converting the img into grayscale
                cropped_img = img_gray[y1:y2, x1:x2]
                cropped_img = noise_remove(cropped_img)
                overlay+=cropped_img

    else:
        print(f'Subdirectory {video_folders} not found')
    if return_base == True:
        return extract_Data(folder_path, img_order, resolutions, overlay), overlay
    else:
        return overlay

def extract_Data(folder_path, img_order, resolutions, overlay, use_threshold = False, reference = None):
    total_fluorescence = [] #empty list to add the max fluoresence later and plot
    subdir_path = os.path.join(cwd,'Videos', folder_path) #create path to directory in the iteration
    if use_threshold == True and reference != None:
        threshold = extract_threshold(folder_path, resolutions, reference)

    if os.path.isdir(subdir_path): #checking if subdirectory path exist
        for img in img_order:
            img_path = os.path.join(subdir_path, img)
            if os.path.exists(img_path): #filter for only .jpg files inside subdir_path
                a=plt.imread(img_path) #need save plt.imread to a variable, if not cannot plot downstream
                img_gray = rgb2gray(a) #converting the img into grayscale
                y1, y2, x1, x2 = resolutions
                cropped_img = img_gray[y1:y2, x1:x2]
                cropped_img = noise_remove(cropped_img)
                if use_threshold == True:
                    cropped_img[cropped_img < threshold] = 0
                cropped_img *= overlay
                total_fluorescence.append(cropped_img.sum()) #sum up signal inside individual frames and append into total_fluorascence
    else:
        print(f'Subdirectory {video_folders} not found')
    return total_fluorescence

def extract_thresholdData(folder_path, img_order, resolutions, reference):
    total_fluorescence = [] #empty list to add the max fluoresence later and plot
    subdir_path = os.path.join(cwd,'Videos', folder_path) #create path to directory in the iteration
    threshold = extract_threshold(folder_path, resolutions, reference)
    y1, y2, x1, x2 = resolutions

    if os.path.isdir(subdir_path): #checking if subdirectory path exist
        for img in img_order:
            img_path = os.path.join(subdir_path, img)
            if os.path.exists(img_path): #filter for only .jpg files inside subdir_path
                a=plt.imread(img_path) #need save plt.imread to a variable, if not cannot plot downstream
                img_gray = rgb2gray(a) #converting the img into grayscale
                cropped_img = img_gray[y1:y2, x1:x2]
                cropped_img[cropped_img<threshold] = 0
                total_fluorescence.append(cropped_img.sum()) #sum up signal inside individual frames and append into total_fluorascence
    else:
        print(f'Subdirectory {video_folders} not found')
    return total_fluorescence

def extract_threshold(folder_path, resolutions, reference):
    ref_path = os.path.join(cwd, 'Videos', folder_path, reference)
    img_ref = plt.imread(ref_path)

    #reference threshold
    img_ref = rgb2gray(img_ref)
    y1, y2, x1, x2 = resolutions
    img_ref_cropped = img_ref[y1:y2, x1:x2]
    threshold = threshold_isodata(img_ref_cropped)
    return threshold

def power_overlay(overlay):
    overlay = overlay**4
    return overlay

def norm_overlay(overlay):
    overlay = overlay/np.max(overlay)
    return overlay

def get_label(filepath):
    # x=list(region_of_interest.keys())[4]
    label=[fragments for fragments in filepath.split('_') if 'drug' in fragments]
    return label[0].upper()

def noise_remove(img):
    mask = img < 0.15
    img[mask] = 0
    return img

In [ ]:
# This cycles through all video folders and creates the graphs
for key in region_of_interest:
    for i in range(len(region_of_interest[key])):
        video1 = extract_fluorescenceData(key, vf_dict[key], region_of_interest[key][i])

        #first overlay without any configuration to heat map
        img_overlay, overlay = overlay_threshold(key, vf_dict[key], region_of_interest[key][i])
        img_overlay = img_overlay/max(img_overlay)

        #increasing the heat map by power of 4, not 2. squared is just an old name and im lazy
        overlay1 = power_overlay(overlay)
        img_squared = extract_Data(key, vf_dict[key], region_of_interest[key][i], overlay1)
        img_squared = img_squared/max(img_squared)

        #this is normalising the heat map, but it does the same as the first function
        overlay2 = norm_overlay(overlay)
        img_norm = extract_Data(key, vf_dict[key], region_of_interest[key][i], overlay2)
        img_norm = img_norm/max(img_norm)

        #fluorescence of ratio
        ratio_f = ratio_Data(key, vf_dict[key], region_of_interest[key][i])
        ratio_f = ratio_f/max(ratio_f)

        img_mask = extract_thresholdData(key, vf_dict[key], region_of_interest[key][i], golgi_frame[key])
        img_mask = img_mask/max(img_mask)

        threshold_weighted = extract_Data(key, vf_dict[key], region_of_interest[key][i], overlay1,
                                          use_threshold = True, reference = golgi_frame[key])
        threshold_weighted = threshold_weighted/max(threshold_weighted)

        frames = np.arange(1,len(img_overlay)+1)
        plt.figure(figsize = (8,4))
        plt.plot(frames, img_overlay, label = "Linear", linestyle = 'solid', marker = '*')
        plt.plot(frames, img_squared, label = "Powered", linestyle = 'solid', marker = 'v')
        plt.plot(frames, ratio_f, label = 'Fluorescence/Area', linestyle = 'solid', marker = 's')
        plt.plot(frames, img_mask, label = 'Threshold', linestyle = 'solid', marker = 'p')
        plt.plot(frames, threshold_weighted, label = "Threshold and Power weight", linestyle = 'solid', marker = 'P')
        histogrammer(video1, key, i, label = True) #reusing previous code
        plt.title(f'{key} Region of Interest {i}')
        plt.legend(loc = 'upper right', bbox_to_anchor = (1.38,1))
        plt.tight_layout()
        plt.savefig(f'{key} Region of Interest {i}', format = "jpg")
        plt.show()

#Assessing the rate of export of TNF
Likely will require data on how fluorescence levels correspond to GFP levels to create a calibration curve.

While we are not able to assess exactly how fast it is exported and express it in terms of concentration per unit time, we can still assess whether the rate of export is linearly related or if other mathematical equations may better suit the rate of export.

In [ ]:
#chosen function: power
def linear_relation(t,m,c):
    export = m*t + c
    return export

def dcdt(t,m,c,f_actual): #vary intercept
    f_predicted = linear_relation(t,m,c)
    a = np.sum(f_actual - f_predicted)
    b = -2/len(f_actual)
    const = a * b
    return const

def dmdt(t,m,c,f_actual): #vary gradient
    f_predicted = linear_relation(t,m,c)
    a = np.sum((f_actual - f_predicted) * t)
    b = -2/len(f_actual)
    gradient = a * b
    return gradient

def get_lineEqn(m, c, t, f_i, max_iter=10000, lr = 1e-4, tol = 1e-5):
    for _ in range(max_iter):
        diff1 = dcdt(t,m,c,f_i)*lr
        diff2 = dmdt(t,m,c,f_i)*lr
        if abs(diff1) < tol and abs(diff2) < tol:
            break
        else:
            c-=diff1
            m-=diff2
    return m,c

def rss(y_predicted, y_actual):
    return np.sum((y_actual-y_predicted)**2)

def tss(y_predicted, y_actual):
    y_ave = np.average(y_actual)
    return np.sum((y_predicted-y_ave)**2)

def r2calc(y1, ya):
    r2 = 1 - (rss(y1,ya)/tss(y1,ya))
    return r2

In [ ]:
# We are calling both scipy's linear regression function, and our own linear
# regression model that is seen in get_lineEqn().
for key in region_of_interest.keys():
    for i in range(len(region_of_interest[key])):
        heat_map = overlay_threshold(key, vf_dict[key], region_of_interest[key][i], return_base = False)
        weighted_fluorescence = extract_Data(key, vf_dict[key], region_of_interest[key][i], power_overlay(heat_map))
        weighted_fluorescence = weighted_fluorescence/np.max(weighted_fluorescence)
        max_index = np.where(weighted_fluorescence == np.max(weighted_fluorescence))
        export = weighted_fluorescence[max_index[0][0]:]
        frames = np.arange(max_index[0][0],len(weighted_fluorescence))
        t = (np.arange(1,len(export)+1))*5
        plt.figure(figsize = (10,4))
        plt.grid(alpha = 0.25)
        plt.plot(frames, export, '.')
        plt.title(f'{key} \nRegion of Interest {i} \n{region_of_interest[key][i]}')

        function = linregress(t,export, alternative = 'less')
        line = linear_relation(t, function.slope, function.intercept)
        plt.plot(frames, line, label = r'export = {:.3f}t + {:.3f}, $r^2$ = {:.3f}'.format(function.slope,function.intercept,function.rvalue**2))

        m, c = get_lineEqn(m=-1,c=1, t=t, f_i = export)
        my_line = linear_relation(t,m,c)
        r2 = r2calc(my_line, export)
        plt.plot(frames, my_line, label = r'export = {:.3f}t + {:.3f}, $r^2$ = {:.3f}'.format(m, c, r2), linestyle = 'dashed')
        plt.legend(loc = 'upper right')
        plt.xlabel('Frame Number')
        plt.ylabel('Relative fluorescence', rotation = 90)
        plt.show()

In [ ]:
# this shows the heat maps
for key in region_of_interest.keys():
    for i in range(len(region_of_interest[key])):
        heat_map = overlay_threshold(key, vf_dict[key], region_of_interest[key][i], return_base = False)
        plt.imshow(heat_map, cmap = 'jet')
        plt.axis('off')
        plt.colorbar()
        plt.show()

In [ ]:
i = 0
key = '0.5ugDNA_drug10uM_5min45min_1'
video1 = extract_fluorescenceData(key, vf_dict[key], region_of_interest[key][i])

#first overlay without any configuration to heat map
img_overlay, overlay = overlay_threshold(key, vf_dict[key], region_of_interest[key][i])
img_overlay = img_overlay/max(img_overlay)

#increasing the heat map by power of 4, not 2. squared is just an old name and im lazy
overlay1 = power_overlay(overlay)
img_squared = extract_Data(key, vf_dict[key], region_of_interest[key][i], overlay1)
img_squared = img_squared/max(img_squared)

#this is normalising the heat map, but it does the same as the first function
overlay2 = norm_overlay(overlay)
img_norm = extract_Data(key, vf_dict[key], region_of_interest[key][i], overlay2)
img_norm = img_norm/max(img_norm)

#fluorescence of ratio
ratio_f = ratio_Data(key, vf_dict[key], region_of_interest[key][i])
ratio_f = ratio_f/max(ratio_f)

img_mask = extract_thresholdData(key, vf_dict[key], region_of_interest[key][i], golgi_frame[key])
img_mask = img_mask/max(img_mask)

threshold_weighted = extract_Data(key, vf_dict[key], region_of_interest[key][i], overlay1,
                                  use_threshold = True, reference = golgi_frame[key])
threshold_weighted = threshold_weighted/max(threshold_weighted)

frames = np.arange(1,len(img_overlay)+1)
plt.figure(figsize = (12,4))
plt.plot(frames, img_overlay, label = "Linear", linestyle = 'solid', marker = '*')
plt.plot(frames, img_squared, label = "Powered", linestyle = 'solid', marker = 'v')
plt.plot(frames, ratio_f, label = 'Fluorescence/Area', linestyle = 'solid', marker = 's')
plt.plot(frames, img_mask, label = 'Threshold', linestyle = 'solid', marker = 'p')
plt.plot(frames, threshold_weighted, label = "Threshold and Power weight", linestyle = 'solid', marker = 'P')
histogrammer(video1, key, label = True) #reusing previous code
plt.title(f'{key} Region of Interest {i}')
plt.legend(loc = 'upper right', bbox_to_anchor = (1.38,1))
plt.tight_layout()
plt.savefig(f'{key} Region of Interest {i}', format = "jpg")
plt.show()